In [210]:
import cv2
import os
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt
import numpy as np
from joblib import Parallel, delayed


In [211]:
def readPascalVoc(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    for boxes in root.iter('object'):

        ymin, xmin, ymax, xmax, name = None, None, None, None, None
        
        
        name = boxes.find('name').text
        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        list_with_single_boxes = (name,[xmin, ymin, xmax, ymax])
        list_with_all_boxes.append(list_with_single_boxes)

    return list_with_all_boxes

In [212]:
pathToDataSet = os.path.join("/home/mortpo/ESIREM/IA/SuperMaskDetector","DataSet")
pathToAnnotation = os.path.join(pathToDataSet,"Annotations")
pathToImg = os.path.join(pathToDataSet,"Images")
fichierAnnotation = os.listdir(pathToAnnotation)

In [213]:
def readImagesFromAnnotation(annotation: str):
    fichierXML = os.path.join(pathToAnnotation,annotation)
    boxes = readPascalVoc(fichierXML)
    fichierImage = os.path.join(pathToImg,annotation.replace(".xml",".jpg"))
    image = None
    if os.path.exists(fichierImage):
        image = cv2.imread(fichierImage)
    else:
        print(fichierImage,"inexistant traitement passé")
        return (None,None,None)
    sizeX,sizeY = image.shape[:2]
    image_with_mask = np.zeros((sizeX,sizeY,1), np.uint8)
    image_without_mask = np.zeros((sizeX,sizeY,1), np.uint8)
    image_with_incorrect_mask = np.zeros((sizeX,sizeY,1), np.uint8)
    for visage in boxes:
        couleur = 255
        point1 = (visage[1][0],visage[1][1])
        point2 = (visage[1][2],visage[1][3])
        if "with_mask" in visage[0]:
            cv2.rectangle(image_with_mask, point1, point2, couleur,-1)

        if "without_mask" in visage[0]:
            cv2.rectangle(image_without_mask, point1, point2, couleur,-1)

        if "with_incorrect_mask" in visage[0]:
            cv2.rectangle(image_with_incorrect_mask, point1, point2, couleur,-1)
    basseresolution = (426,240)
    image_with_mask= cv2.resize(image_with_mask,basseresolution,interpolation = cv2.INTER_NEAREST)
    image_without_mask =cv2.resize(image_without_mask,basseresolution,interpolation = cv2.INTER_NEAREST)
    image_with_incorrect_mask =cv2.resize(image_with_incorrect_mask,basseresolution,interpolation = cv2.INTER_NEAREST)
    return (image_with_mask,image_without_mask,image_with_incorrect_mask)
    

In [214]:
def writeImage(annotation: str):
    image_with_mask,image_without_mask,image_with_incorrect_mask = readImagesFromAnnotation(annotation)
    nomfichier = annotation.replace(".xml","")
    path_image_with_mask = os.path.join(pathToDataSet,"TRAINING","with_mask",nomfichier+".jpg")
    path_image_without_mask = os.path.join(pathToDataSet,"TRAINING","without_mask",nomfichier+".jpg")
    path_image_with_incorrect_mask= os.path.join(pathToDataSet,"TRAINING","with_incorrect_mask",nomfichier+".jpg")
    cv2.imwrite(path_image_with_mask, image_with_mask)
    cv2.imwrite(path_image_without_mask, image_without_mask)
    cv2.imwrite(path_image_with_incorrect_mask, image_with_incorrect_mask)
    return nomfichier

    

In [215]:
def showAnnotation(annotation: str):
    image_with_mask,image_without_mask,image_with_incorrect_mask = readImagesFromAnnotation(annotation)

    plt.figure(figsize=(120, 80))
    ax = plt.subplot(1, 4, 1)
    plt.imshow(image_with_mask,cmap=plt.cm.gray)
    ax = plt.subplot(1, 4, 2)
    plt.imshow(image_without_mask,cmap=plt.cm.gray)
    ax = plt.subplot(1, 4, 3)
    plt.imshow(image_with_incorrect_mask,cmap=plt.cm.gray)

    plt.show()

In [217]:
Parallel(n_jobs=8)(delayed(writeImage)(fichierAnnotation[i]) for i in range(len(fichierAnnotation)))

['8e874bd80319997ba26404bdae37b82e',
 '4abf0c479e92f16e31de4de0e1ff387e',
 '245a9bbcc35da8979740f9067c805a5b',
 '6f29558ac2de1c7c6f769722f9754997',
 '73e7beeebe6378cd8f6f2244e11b30dd',
 '9f9fc30acd75904a4b6fa5c37434cc16',
 '20d47c24835aa3cf467e90d9aec0a011',
 '30c4403d4c6e3ad905c483c95dc4183a',
 '7cd23b1fc84836e0e0d4dd3b453555fa',
 '61a0fb594843c6aa1fd379135c0e425d',
 '39eed0b45c59c2bacbc0413f38be5c0c',
 '2dadb438128920b0d471d8bfaeaebcc5',
 '77c355531024acf8b229db2600b0d9af',
 '9b1b583add26610d44e84e8a2857c915',
 '74ea414d53833621bb3b2cae6c64fa0b',
 '4fe5bb9eedae0752258c110fb5eea8ba',
 '7d98cb9f9ad283ee96d4c960178becc5',
 '86e70720fdf45dd0ceca2e02a3f82229',
 '251bfedd3d36f3cfb34a4d09c8666607',
 '5d3d10e4f03718ee97bf6f5cde9271ab',
 '58c3f47c8c0998511f94cd414fcc4d74',
 '3eeb217ecfa91cbfc1b7c79e72bb0aa5',
 '58cfcb991239c2a43d88a0d7db1a9cc0',
 '4f4893e7aa4e5d3491ad000b490503d0',
 '4be121c52c1bfcf42c13d81ed4ab8c66',
 '3b99eefb679d64902d940b82a53f8371',
 '7e02fc700086e38fe1fdf310f3841011',
 

In [218]:
"""fichannot = os.listdir(pathToAnnot)
fichimg = os.listdir(pathToImg)
annotnom = set()
imgnom = set()
[annotnom.add(x.replace(".xml","")) for x in fichannot]
[imgnom.add(x.replace(".jpg","")) for x in fichimg]
print(annotnom - imgnom)"""

'fichannot = os.listdir(pathToAnnot)\nfichimg = os.listdir(pathToImg)\nannotnom = set()\nimgnom = set()\n[annotnom.add(x.replace(".xml","")) for x in fichannot]\n[imgnom.add(x.replace(".jpg","")) for x in fichimg]\nprint(annotnom - imgnom)'